In [ ]:
import os
import numpy as np
import pandas as pd
import pandas_profiling
import plotnine
from plotnine import *  # Provides a ggplot-like interface to matplotlib.
from IPython.display import display


In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd

In [ ]:
import pandas
import os

# This query represents dataset "full_features" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_77300097_person_sql = """
    SELECT
        person.person_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        p_race_concept.concept_name as race,
        p_ethnicity_concept.concept_name as ethnicity,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (
                                    SELECT
                                        DISTINCT ca.descendant_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                                    JOIN
                                        (
                                            select
                                                distinct c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (715939) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) b 
                                                    ON (
                                                        ca.ancestor_id = b.concept_id
                                                    )
                                            ) 
                                            AND is_standard = 1
                                        )
                                ) criteria 
                        GROUP BY
                            criteria.person_id,
                            criteria.concept_id 
                        HAVING
                            COUNT(criteria.person_id) >= 3 
                        ) 
                    )"""

person_df = pandas.read_gbq(
    dataset_77300097_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

person_df.head(5)

In [ ]:
import pandas
import os

# This query represents dataset "full_features" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v7
dataset_77300097_measurement_sql = """
    SELECT
        measurement.person_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        m_type.concept_name as measurement_type_concept_name,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        m_value.concept_name as value_as_concept_name,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
        WHERE
            (
                measurement_source_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    1586218, 903107, 903111, 903115, 903118, 903121, 903124, 903126, 903133, 903135, 903136
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 0 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    measurement.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT ca.descendant_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                                                JOIN
                                                    (
                                                        select
                                                            distinct c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    concept_id IN (715939) 
                                                                    AND full_text LIKE '%_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                is_standard = 1 
                                                                AND is_selectable = 1
                                                            ) b 
                                                                ON (
                                                                    ca.ancestor_id = b.concept_id
                                                                )
                                                        ) 
                                                        AND is_standard = 1
                                                    )
                                            ) criteria 
                                    GROUP BY
                                        criteria.person_id,
                                        criteria.concept_id 
                                    HAVING
                                        COUNT(criteria.person_id) >= 3 
                                    ) 
                                ))) measurement 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
                                ON measurement.measurement_concept_id = m_standard_concept.concept_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_type 
                                ON measurement.measurement_type_concept_id = m_type.concept_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_operator 
                                ON measurement.operator_concept_id = m_operator.concept_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_value 
                                ON measurement.value_as_concept_id = m_value.concept_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_unit 
                                ON measurement.unit_concept_id = m_unit.concept_id 
                        LEFT JOIn
                            `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                ON measurement.visit_occurrence_id = v.visit_occurrence_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_visit 
                                ON v.visit_concept_id = m_visit.concept_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_source_concept 
                                ON measurement.measurement_source_concept_id = m_source_concept.concept_id"""

measurement_df = pandas.read_gbq(
    dataset_77300097_measurement_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

measurement_df.head(5)

In [ ]:
import pandas
import os

# This query represents dataset "full_features" for domain "observation" and was generated for All of Us Controlled Tier Dataset v7
dataset_77300097_observation_sql = """
    SELECT
        observation.person_id,
        o_standard_concept.concept_name as standard_concept_name,
        o_standard_concept.vocabulary_id as standard_vocabulary,
        observation.observation_datetime,
        o_type.concept_name as observation_type_concept_name,
        observation.value_as_number,
        observation.value_as_string,
        o_value.concept_name as value_as_concept_name,
        o_qualifier.concept_name as qualifier_concept_name,
        o_unit.concept_name as unit_concept_name,
        o_visit.concept_name as visit_occurrence_concept_name,
        observation.observation_source_value,
        o_source_concept.concept_name as source_concept_name,
        o_source_concept.vocabulary_id as source_vocabulary,
        observation.unit_source_value,
        observation.qualifier_source_value,
        observation.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
        WHERE
            (
                observation_concept_id IN (
                    4139934
                )
            )  
            AND (
                observation.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT ca.descendant_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                                            JOIN
                                                (
                                                    select
                                                        distinct c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (715939) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) b 
                                                            ON (
                                                                ca.ancestor_id = b.concept_id
                                                            )
                                                    ) 
                                                    AND is_standard = 1
                                                )
                                        ) criteria 
                                GROUP BY
                                    criteria.person_id,
                                    criteria.concept_id 
                                HAVING
                                    COUNT(criteria.person_id) >= 3 
                                ) 
                            ))) observation 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_standard_concept 
                            ON observation.observation_concept_id = o_standard_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_type 
                            ON observation.observation_type_concept_id = o_type.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_value 
                            ON observation.value_as_concept_id = o_value.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_qualifier 
                            ON observation.qualifier_concept_id = o_qualifier.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_unit 
                            ON observation.unit_concept_id = o_unit.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                            ON observation.visit_occurrence_id = v.visit_occurrence_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_visit 
                            ON v.visit_concept_id = o_visit.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_source_concept 
                            ON observation.observation_source_concept_id = o_source_concept.concept_id"""

observation_df = pandas.read_gbq(
    dataset_77300097_observation_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

observation_df.head(5)

In [ ]:
import pandas
import os

# This query represents dataset "full_features" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_77300097_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_type.concept_name as condition_type_concept_name,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_source_concept.vocabulary_id as source_vocabulary 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    317009, 4193704, 4282316, 437264, 442077, 80180
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT ca.descendant_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                                                JOIN
                                                    (
                                                        select
                                                            distinct c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    concept_id IN (715939) 
                                                                    AND full_text LIKE '%_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                is_standard = 1 
                                                                AND is_selectable = 1
                                                            ) b 
                                                                ON (
                                                                    ca.ancestor_id = b.concept_id
                                                                )
                                                        ) 
                                                        AND is_standard = 1
                                                    )
                                            ) criteria 
                                    GROUP BY
                                        criteria.person_id,
                                        criteria.concept_id 
                                    HAVING
                                        COUNT(criteria.person_id) >= 3 
                                    ) 
                                ))) c_occurrence 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                                ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                                ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                                ON v.visit_concept_id = visit.concept_id 
                        LEFT JOIN
                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                                ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id"""

condition_df = pandas.read_gbq(
    dataset_77300097_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

condition_df.head(5)

In [ ]:
def cleanup_names(column, keywords):
    """
    Replaces various medication names with their root medication names.

    Args:
    column (pd.Series): The pandas column containing medication names.

    Returns:
    pd.Series: The modified pandas column with root medication names.
    """
    for keyword in keywords:
        column = column.str.replace(rf'.*{keyword}.*', keyword, regex=True)
    return column

In [ ]:
list(measurement_df)

In [ ]:
measurement_df[measurement_df['person_id'] == 1834253]

In [ ]:
measurement_df['measurement_source_value'].unique()

In [ ]:
def transform_measurement_df(df):
    """
    Transforms the measurement DataFrame into a new format with specific columns for each measurement type.
    Uses 'value_as_number' unless 'value_as_concept_name' is not 'No matching concept', 
    then it uses 'value_as_concept_name'. Includes unit columns only for measurements derived from 'value_as_number'.

    Args:
    df (pd.DataFrame): DataFrame containing patient measurement data.

    Returns:
    pd.DataFrame: Transformed DataFrame with specified columns and units.
    """
    # Define the measurement types and their corresponding source value columns
    measurements = {
        'wheelchair-user-status': 'wheelchair-user-status',
        'weight': 'weight',
        'heart-rhythm-status': 'heart-rhythm-status',
        'heart-rate-mean': 'heart-rate-mean',
        'waist-circumference-mean': 'waist-circumference-mean',
        'hip-circumference-mean': 'hip-circumference-mean',
        'blood-pressure-systolic-mean': 'blood-pressure-systolic-mean',
        'blood-pressure-mean': 'blood-pressure-mean',
        'height': 'height',
        'blood-pressure-diastolic-mean': 'blood-pressure-diastolic-mean',
        'bmi': 'bmi'
    }

    # Initialize an empty DataFrame to store the transformed data
    transformed_df = pd.DataFrame()

    # Loop through each measurement type
    for measurement, source_value in measurements.items():
        # Filter the rows for the current measurement
        filtered_df = df[df['measurement_source_value'] == source_value]

        # Determine the value based on 'value_as_number' or 'value_as_concept_name'
        filtered_df[measurement] = filtered_df.apply(
            lambda row: row['value_as_number'] 
                        if pd.isna(row['value_as_concept_name']) or row['value_as_concept_name'] == 'No matching concept' 
                        else row['value_as_concept_name'], 
            axis=1
        )

        # Add the unit column for the current measurement only if the value comes from 'value_as_number'
        if filtered_df[measurement].dtype in [float, int]:
            filtered_df[measurement + '-unit'] = filtered_df['unit_concept_name']

        # Select columns to merge
        columns_to_merge = ['person_id', measurement]
        if measurement + '-unit' in filtered_df.columns:
            columns_to_merge.append(measurement + '-unit')

        # Merge the data into the transformed DataFrame
        if transformed_df.empty:
            transformed_df = filtered_df[columns_to_merge].copy()
        else:
            transformed_df = transformed_df.merge(filtered_df[columns_to_merge], 
                                                  on='person_id', 
                                                  how='outer')

    return transformed_df




In [ ]:
transformed_measurement_df = transform_measurement_df(measurement_df)
transformed_measurement_df.head()

In [ ]:
transformed_measurement_df.head()

In [ ]:
def aggregate_conditions(df):
    """
    Aggregates all unique conditions (standard_concept_name) for each person_id into a single list.
    The conditions are sorted alphabetically.

    Args:
    df (pd.DataFrame): DataFrame containing condition data.

    Returns:
    pd.DataFrame: Transformed DataFrame with columns [person_id, all_disorders],
                  where all_disorders is a list of unique, alphabetically sorted conditions.
    """
    # Grouping by person_id and aggregating unique conditions
    aggregated_conditions = df.groupby('person_id')['standard_concept_name'].agg(lambda x: sorted(set(x)))

    # Converting the aggregated data into a DataFrame
    result_df = aggregated_conditions.reset_index(name='all_disorders')

    return result_df

aggregated_condition_df = aggregate_conditions(condition_df)
print(aggregated_condition_df.head())



In [ ]:
temp_merge = aggregated_condition_df.merge(transformed_measurement_df, how='left', on='person_id')

In [ ]:
temp_merge.head()

In [ ]:
clean_temp_merge = temp_merge.drop([
    "blood-pressure-mean",
    "blood-pressure-mean-unit",  
], axis=1)

In [ ]:
clean_temp_merge['wheelchair-user-status'] = clean_temp_merge['wheelchair-user-status'].fillna('No')

In [ ]:
full_merge = person_df.merge(clean_temp_merge, how='left', on='person_id')

In [ ]:
full_merge.info()

In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd




In [ ]:
# This snippet assumes that you run setup first

# This code lists objects in your Google Bucket

# Get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# List objects in the bucket
print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))




In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = full_merge   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'complete_feature_set.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr


In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd




In [ ]:
# This snippet assumes you run setup first

# This code copies file in your Google Bucket and loads it into a dataframe

# Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
name_of_file_in_bucket = 'gs://fc-secure-f691f9bf-9814-462b-be62-53c8dff2e698/data/complete_feature_set.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
feature_set_df = pd.read_csv(name_of_file_in_bucket)


In [ ]:
# This snippet assumes you run setup first

# This code copies file in your Google Bucket and loads it into a dataframe

# Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
name_of_file_in_bucket = 'gs://fc-secure-f691f9bf-9814-462b-be62-53c8dff2e698/data/patient_success.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
success_set_df = pd.read_csv(name_of_file_in_bucket)


In [ ]:
#final_df = success_set_df.merge(feature_set_df, how='left', on='person_id')
final_df.info()

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = final_df   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'full_data_set.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr


In [ ]:
# This snippet assumes that you run setup first

# This code lists objects in your Google Bucket

# Get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# List objects in the bucket
print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))


